In [14]:
import numpy as np
import pickle
import cv2, os
from glob import glob
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras import backend as K
# K.backend.image_data_format('tf')

In [15]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [16]:
def get_image_size():
    img = cv2.imread('gestures/1/100.jpg', 0)
#     print(img.shape)
    return img.shape
get_image_size()

(50, 50)

In [17]:
def get_num_of_classes():
#     print(len(glob('gestures/*')))
    return len(glob('gestures/*'))
get_num_of_classes()

44

In [18]:
image_x, image_y = get_image_size()

## Applying CNN on training data . 

In [19]:
def cnn_model():
    num_of_classes = get_num_of_classes()
    model = Sequential()
    model.add(Conv2D(16, (2,2), input_shape=(image_x, image_y, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3), padding='same'))
    model.add(Conv2D(64, (5,5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_of_classes, activation='softmax'))
    sgd = optimizers.SGD(lr=1e-2)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    filepath="cnn_model_keras3.h5"
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint1]
    from keras.utils import plot_model
    plot_model(model, to_file='model.png', show_shapes=True)
    return model, callbacks_list

In [20]:
def train():
    with open("train_images", "rb") as f:
        train_images = np.array(pickle.load(f))
    with open("train_labels", "rb") as f:
        train_labels = np.array(pickle.load(f), dtype=np.int32)

    with open("val_images", "rb") as f:
        val_images = np.array(pickle.load(f))
    with open("val_labels", "rb") as f:
        val_labels = np.array(pickle.load(f), dtype=np.int32)

    train_images = np.reshape(train_images, (train_images.shape[0], image_x, image_y, 1))
    val_images = np.reshape(val_images, (val_images.shape[0], image_x, image_y, 1))
    train_labels = np_utils.to_categorical(train_labels)
    val_labels = np_utils.to_categorical(val_labels)

    print(val_labels.shape)

    model, callbacks_list = cnn_model()
    model.summary()
    model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=30, batch_size=500, callbacks=callbacks_list)
    scores = model.evaluate(val_images, val_labels, verbose=0)
    print("CNN Error: %.2f%%" % (100-scores[1]*100))
    model.save('cnn_model_keras3.h5')

In [21]:
train()
K.clear_session();

(8800, 44)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 49, 16)        80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 25, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          51264     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 1, 64

/home/rupamay/anaconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/30
176/176 [==============================] - 29s 161ms/step - loss: 3.1835 - accuracy: 0.2277 - val_loss: 0.8342 - val_accuracy: 0.8332
Epoch 2/30
176/176 [==============================] - 29s 163ms/step - loss: 0.5891 - accuracy: 0.8182 - val_loss: 0.0363 - val_accuracy: 0.9937
Epoch 3/30
176/176 [==============================] - 29s 163ms/step - loss: 0.1881 - accuracy: 0.9398 - val_loss: 0.0159 - val_accuracy: 0.9964
Epoch 4/30
176/176 [==============================] - 28s 159ms/step - loss: 0.1113 - accuracy: 0.9645 - val_loss: 0.0082 - val_accuracy: 0.9978
Epoch 5/30
176/176 [==============================] - 28s 160ms/step - loss: 0.0776 - accuracy: 0.9753 - val_loss: 0.0056 - val_accuracy: 0.9989
Epoch 6/30
176/176 [==============================] - 28s 159ms/step - loss: 0.0583 - accuracy: 0.9814 - val_loss: 0.0043 - val_accuracy: 0.9990
Epoch 7/30
176/176 [==============================] - 28s 162ms/step - loss: 0.0471 - accuracy: 0.9851 - val_loss: 0.0028 - val_ac

176/176 [==============================] - 31s 174ms/step - loss: 0.0100 - accuracy: 0.9967 - val_loss: 2.3406e-04 - val_accuracy: 0.9999
Epoch 28/30
176/176 [==============================] - 30s 173ms/step - loss: 0.0089 - accuracy: 0.9973 - val_loss: 6.2968e-05 - val_accuracy: 1.0000
Epoch 29/30
176/176 [==============================] - 30s 173ms/step - loss: 0.0079 - accuracy: 0.9973 - val_loss: 1.4161e-04 - val_accuracy: 1.0000
Epoch 30/30
176/176 [==============================] - 31s 177ms/step - loss: 0.0080 - accuracy: 0.9973 - val_loss: 7.2962e-05 - val_accuracy: 1.0000
CNN Error: 0.00%


## Applying SVC Machine Learning Algorithm on training data . 

In [9]:
from sklearn.svm import LinearSVC
import numpy as np
import pickle
import cv2

svc=LinearSVC(max_iter=1200000)



with open("train_images", "rb") as f:
    train_images = np.array(pickle.load(f))
with open("train_labels", "rb") as f:
    train_labels = np.array(pickle.load(f), dtype=np.int32)

with open("val_images", "rb") as f:
    val_images = np.array(pickle.load(f))
with open("val_labels", "rb") as f:
    val_labels = np.array(pickle.load(f), dtype=np.int32)
print(train_images.shape)
train_images = np.reshape(train_images, (88000,2500))
val_images = np.reshape(val_images, (8800,2500))
# train_labels = np_utils.to_categorical(train_labels)
# train_labels=train_labels.flatten()
# val_labels = np_utils.to_categorical(val_labels)

print(train_labels.shape)

model_SVC=svc.fit(train_images,train_labels)


(88000, 50, 50)
(88000,)


In [ ]:
from sklearn.metrics import accuracy_score

# print(accuracy_score(pred,train_labels)*100)

In [11]:

with open("val_images", "rb") as f:
    val_images = np.array(pickle.load(f))
val_images = np.reshape(val_images, (8800,2500))
pred1=model_SVC.predict(val_images)
print(accuracy_score(pred1,val_labels)*100)

99.98863636363636


In [12]:
with open("svc", "wb") as f:
        pickle.dump(model_SVC, f)

In [13]:
with open("svc", "rb") as g:
        hist = pickle.load(g)